In [139]:
import sys
import os
import glob
home_folder = os.path.dirname(os.getcwd())
sys.path.append(home_folder)

from src.xml_server_connect import *
#from src.xml_report_process import *
from src.utility import *
from src.extract_table import *
from src.pre_process import *
from collections import defaultdict
import pandas as pd
import re
import ftfy
import string
import unicodedata
import calendar
from multiprocessing import Pool
import json
import xml.etree.cElementTree as ET
import lxml.etree
import nltk
from nltk.parse import CoreNLPParser

import unicodedata
import calendar
import datetime

In [2]:
pos_tagger = CoreNLPParser(url='http://localhost:8080', tagtype='pos')
logging.getLogger().setLevel(logging.INFO)

In [3]:
result_folder = '/mnt/results'
dataset_folder = '/domino/datasets/raw_report'

In [4]:
    risk_list = [r'(?:sources*|natures*) of (?:\w*? ){0,2}(?:vulnerabilit(?:y:ies)|impacts*|threats*|risks*|events*|shocks*)', \
                 r'^shocks*$', r'sources* and', r'main risks* to baseline scenario',\
                 r'^risks*$', r'^risks* \d{0,2}(?:,* )*\d{0,2}/*$', r'risk downside',\
                 r'^main threats*$', \
                 r'^events*$', r'^spillover risks$', r'^sources*$', r'^descriptions*$',\
                 r'shocks and likelihood', r'^vulnerabilit(?:y|ies)$', r'sources*/(?:\w*/)*(?:\w* ){0,3}of risks*',\
                 r'tighter or more volatile global financial conditions', \
                 r'risks* overall level of concern']
    likelihood_list = [r'likelihood', r'li kelihood', r'levels* of concerns*', \
                       r'^prob(?:\.|ability|abilities)$', r'like-lihood',\
                       r'probabilit(?:y|ies) of occurrence', r'^relative$', \
                       r'relative likelihoo d', r'relativelikelihood']
    impact_list = [r'expected (?:\w*? )*impacts*', r'possible (?:\w*? )*impacts*', r'impacts* on', r'potential impacts*', \
                   r'impacts* \(*if realized', r'relative impact', r'expected effects*', r'^impacts*$', \
                   r'possible impact', r'impacts* if', r'impact \(low, medium, or high\)', r'^impacts* \d{0,2}(?:,* )*\d{0,2}/*$', \
                   r'impacts* and', r'severity', r'expected im pact', r'^effects*$', r'^rating$']
    policy_response_list = [r'Possible Policy Options',r'policy responses*', r'policy recommendations*', r'policy advices*', r'policy actions*',\
                            r'possible remedial polic(?:y|ies)', r's preparedness', r'policy measures*'\
                            r'polic(?:y|ies) to (?:minimize|reduce|mitigate|mitigating|ameliorate) (?:impacts*|risks*|threats*)',\
                            r'(?:mitigating|mitigation) measures*', r'^recommendations*$', r'recommended polic(?:y|ies)', ]
    channel_list = [r'transmission channels*', r'transmissions*', r'impacts* channels*', r'^channels*$', r'channel mechanism']
    direction_list = [r'direction', r'up/downside', r'^up */ *down(?:-| )*sides*$', r'^up/down$', r'^down */ *up(?:-| )*sides*$']
    horizon_list = [r'timing', r'horizon', r'timeframe']
    skip_title_list = [r'^switzerland$', r'^overall level of concern$', r'^domestic(?: risks*)*$', r'risks* asses{1,2}ments* matrix',\
                       r'^external(?: risks*)*$', r'switzerland', '^global(?: risks*)*$', r'^sides*$', \
                       r'^other$', r'^fiscal$', r'^(red= high; green=low)$', r'^monetary/exchange rate$', \
                       r'^(red= high likelihood; green = low likelihood)$', r'^financial$',\
                       r'^(red= high; green=low)', r'(red= high likelihood; green = low likelihood)$',\
                       r'potential deviations* from baselines*',\
                       r'country[- ]specific risks*']
    descriptive_list = [r'^description$', ]
    content_list = [r'tighter or more volatile global financial conditions:']

    table_head_pattern = re.compile('|'.join(TABLE_HEAD_PATTERN_LIST))
    risk_pattern = re.compile('|'.join(risk_list), re.I)
    likelihood_pattern = re.compile('|'.join(likelihood_list), re.I)
    impact_pattern = re.compile('|'.join(impact_list), re.I)
    policy_response_pattern = re.compile('|'.join(policy_response_list), re.I)
    channel_pattern = re.compile('|'.join(channel_list), re.I)
    direction_pattern = re.compile('|'.join(direction_list), re.I)
    horizon_pattern = re.compile('|'.join(horizon_list), re.I)
    content_pattern = re.compile('|'.join(content_list), re.I)
    skip_pattern = re.compile('|'.join(skip_title_list), re.I)

In [6]:
##Get the number of total columns of table
def get_col_num(table):
    col_num = 0
    table_rows = table.findall(".//tr")
    for tr in table_rows:
        tds = tr.findall(".//td")
        tds.extend(tr.findall(".//th"))
        cur_col_num = 0
        for td in tds:
            col_span = int(td.attrib['colspan']) if 'colspan' in td.attrib.keys() else 1
            cur_col_num += col_span
        col_num = max(col_num, cur_col_num)
    return col_num

In [7]:
##Fill out all the empty cells in tables by spliting cells whose rowspan is more than 1. Then all rows have the same
##number of columns.
def extract_table(table_list):
    tbody_rows = table_list[0].findall(".//tbody/tr")
    if len(tbody_rows) ==0:
        tbody_rows = table_list[0].findall(".//tr")
        
    td_list = []
    for tr in tbody_rows:
        td = tr.findall(".//td")
        td_list.append(td)

    col_num = get_col_num(table_list[0])
    for n in td_list:
        while len(n) < col_num:
            n.append('')

    for i in range(col_num):
        for n in range(len(td_list)):
            if n > 0 and td_list[n][i] == td_list[n-1][i]:
                continue
            if hasattr(td_list[n][i], 'attrib') is True:
                rowspan = int(td_list[n][i].attrib['rowspan']) if 'rowspan' in td_list[n][i].attrib.keys() else 1
            else:
                rowspan = 0
            x = 1
            while rowspan > 1:
                index = td_list[n].index(td_list[n][i])
                td_list[n+x].insert(index,td_list[n][i])
                rowspan = rowspan - 1
                x = x + 1
        
    for n in td_list:
        n[:] = (value for value in n if value != '')
    totallist=[]
    for n in td_list:
        sublist=[]
        for cell in n:
            text = ' '.join(cell.itertext())
            sublist.append(text)
        totallist.append(sublist)
        
    df = pd.DataFrame.from_records(totallist)
    return df

In [134]:
##One special table format
def extract_table_special(table_list):
    tbody_rows = table_list[0].findall(".//tbody/tr")
    td_list = []
    for tr in tbody_rows:
        td = tr.findall(".//td")
        td_list.append(td)
        
    col_num = get_col_num(table_list[0])
    for n in td_list:
        while len(n) < col_num:
            n.append('')
    
    for i in range(col_num):
        for n in range(len(td_list)):
            if n > 0 and td_list[n][i] == td_list[n-1][i]:
                continue
            if hasattr(td_list[n][i], 'attrib') is True:
                rowspan = int(td_list[n][i].attrib['rowspan']) if 'rowspan' in td_list[n][i].attrib.keys() else 1
                if rowspan == 5:
                    rowspan = rowspan - 2
            else:
                rowspan = 0
            x = 1
            while rowspan > 1:
                index = td_list[n].index(td_list[n][i])
                td_list[n+x].insert(index,td_list[n][i])
                rowspan = rowspan - 1
                x = x + 1
    
    for n in td_list:
        n[:] = (value for value in n if value != '')
    totallist=[]
    for n in td_list:
        sublist=[]
        for cell in n:
            text = ' '.join(cell.itertext())
            sublist.append(text)
        totallist.append(sublist)
        
    df = pd.DataFrame.from_records(totallist)
    return df

In [34]:
def new_boundary_case_thead(country, year, colname2id, id2colname, skipid, contents):      
    indicator = False
    if country == 'china' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['direction'] = [2]
        colname2id['likelihood'] = [3]
        colname2id['impact'] = [4]
        colname2id['policy'] = [5]
        id2colname[0] = ['risk']
        id2colname[2] = ['direction']
        id2colname[3] = ['likelihood']
        id2colname[4] = ['impact']
        id2colname[5] = ['policy']
        skipid.add(1)
        indicator = True
    if (country == 'brazil' and year == 2012) or (country == 'guatemala' and (year == 2013 or year == 2014))\
        or (country == 'el salvador' and year == 2013) or (country == 'colombia' and year == 2013):
        colname2id['risk'] = [0]
        colname2id['direction'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['direction']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True
    if country == 'sweden' and year == 2012:
        colname2id['risk'] = [1]
        colname2id['likelihood'] = [3]
        colname2id['impact'] = [4]
        colname2id['policy'] = [5]
        id2colname[1] = ['risk']
        id2colname[3] = ['likelihood']
        id2colname[4] = ['impact']
        id2colname[5] = ['policy']
        indicator = True
    if (country == 'japan' and (year == 2013 or year == 2014 or year == 2015 or year == 2016)) \
        or (country == 'hong kong' and year == 2016)\
        or (country == 'micronesia' and year == 2015):
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact', 'policy']
        indicator = True
    if country == 'colombia' and year == 2014:
        colname2id['risk'] = [0, 1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True
    if country == 'myanmar' and year == 2015:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['channel'] = [2]
        colname2id['impact'] = [3, 4]
        colname2id['policy'] = [5]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['channel']
        id2colname[3] = ['impact']
        id2colname[4] = ['impact']
        id2colname[5] = ['policy']
        indicator = True
    if (country == 'peru' and (year == 2015 or year == 2016 or year == 2017 or year == 2018)) or (country == 'ecuador' and year == 2015) \
        or (country == 'united kingdom' and year == 2013) or (country == 'colombia' and year == 2015)\
        or (country == 'south sudan' and year == 2014) or (country == 'slovak' and (year == 2014 or year == 2016)):
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'chile' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['horizon'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['horizon']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        indicator = True
    if country == 'peru' and year == 2014:
        colname2id['horizon'] = [0]
        colname2id['risk'] = [1] 
        colname2id['direction'] = [2]
        colname2id['likelihood'] = [3]
        colname2id['impact'] = [4]
        colname2id['policy'] = [5]
        id2colname[0] = ['horizon']
        id2colname[1] = ['risk']
        id2colname[2] = ['direction']
        id2colname[3] = ['likelihood']
        id2colname[4] = ['impact']
        id2colname[5] = ['policy']
        indicator = True
    if (country == 'libya' and year == 2013) or (country == 'denmark' and year == 2013):
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1] 
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'tunisia' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['direction'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['direction']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact', 'policy']
        indicator = True
    if country == 'uruguay' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['direction'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['direction']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        indicator = True
    if country == 'hong kong' and (year == 2017 or year == 2018):
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        colname2id['policy'] = [2]
        id2colname[0] = ['likelihood']
        id2colname[1] = ['impact']
        id2colname[2] = ['policy']
        indicator = True
    if country == 'cambodia' and (year == 2013 or year == 2014):
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        id2colname[0] = ['risk', 'likelihood']
        id2colname[1] = ['impact']
        indicator = True
    if country == '' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['direction'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['channel'] = [4]
        colname2id['policy'] = [5]
        id2colname[0] = ['risk']
        id2colname[1] = ['direction']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['channel']
        id2colname[5] = ['policy']
        indicator = True
    if country == 'iran' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['horizon'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['horizon']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True
    if country == 'maldives' and year == 2017 :
        colname2id['risk'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[1] = ['risk']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'bangladesh' and year == 2018:
        colname2id['risk'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[1] = ['risk']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'russia' and year == 2015:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'bulgaria' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True   
    if country == 'south sudan' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True      
    if country == 'costa rica' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'nicaragua' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True 
    if country == 'poland' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'guatemala' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True 
    if country == 'costa rica' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'costa rica' and year == 2015:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'nicaragua' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True 
    if country == 'st. vincent and the grenadines' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2,3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True 
    if country == 'st. vincent and the grenadines' and year==2015:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2,3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True 
    if country == 'belarus' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True   
    if country == 'belarus' and year == 2013:
        colname2id['risk'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [5]
        id2colname[1] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[5] = ['policy']
        indicator = True
    if country == 'lebanon' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True
    if country == 'algeria' and year == 2014:
        colname2id['risk'] = [0,1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        indicator = True
    if country == 'nicaragua' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['channel'] = [4]
        colname2id['policy'] = [5]
        id2colname[0] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['channel']
        id2colname[5] = ['policy']
        indicator = True
    if country == 'chad' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact','policy']
        indicator = True
    if country == 'san marino' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk','likelihood']
        id2colname[1] = ['impact']
        id2colname[2] = ['policy']
        indicator = True
    
    if country == 'new zealand' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'south korea' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'montenegro' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'paraguay' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'austria' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'lesotho' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'brazil' and year == 2013:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'israel' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        id2colname[0] = ['risk','likelihood']
        id2colname[1] = ['impact']
        indicator = True
    if country == 'switzerland' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'belgium' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk','likelihood']
        id2colname[1] = ['impact']
        id2colname[2] = ['policy']
        indicator = True
    if country == 'st. vincent and the grenadines' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'united kingdom' and year == 2012:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'st. kitts and nevis' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    if country == 'romania' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['channel'] = [1]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood','channel']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'australia' and year == 2017:
        colname2id['risk'] = [1]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [4]
        colname2id['policy'] = [5]
        id2colname[1] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[4] = ['impact']
        id2colname[5] = ['policy']
        indicator = True
    if country == 'san marino' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'togo' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2,3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['impact']
        indicator = True
    if country == 'somalia' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    
    if country == 'latvia' and year == 2016:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [0]
        colname2id['impact'] = [1]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk','likelihood']
        id2colname[1] = ['impact']
        id2colname[2] = ['policy']
        indicator = True
    if country == 'thailand' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [3]
        id2colname[0] = ['risk']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        id2colname[3] = ['policy']
        indicator = True
    if country == 'guatemala' and year == 2014:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [2]
        colname2id['impact'] = [3]
        colname2id['policy'] = [4]
        id2colname[0] = ['risk']
        id2colname[2] = ['likelihood']
        id2colname[3] = ['impact']
        id2colname[4] = ['policy']
        indicator = True
    if country == 'malta' and year == 2017:
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [1]
        colname2id['impact'] = [2]
        colname2id['policy'] = [0]
        id2colname[0] = ['risk','policy']
        id2colname[1] = ['likelihood']
        id2colname[2] = ['impact']
        indicator = True
    
    return  colname2id, id2colname, skipid, contents, indicator

In [10]:
def Insert_row(row_number, df, row_value):  
    start_upper = 0
    end_upper = row_number 
    start_lower = row_number 
    end_lower = df.shape[0] 
    upper_half = [*range(start_upper, end_upper, 1)] 
    lower_half = [*range(start_lower, end_lower, 1)] 
    lower_half = [x.__add__(1) for x in lower_half]  
    index_ = upper_half + lower_half 
    df.index = index_ 
    df.loc[row_number] = row_value  
    df = df.sort_index() 
    return df

In [317]:
skip_row = ['^Global Shocks$','^Georgia-Specific Risks$','^Potential Domestic Shocks$','^Potential External Shocks$',
'^External Risks and Spillovers$','^External Risks$','^Domestic Risks$','^Domestic risks$','^Global risks$','^Country-specific risks$',
'^Domestic$','^Global Risks$','^Domestic risks$','^Global risks$','^Policy Response$','^External risks$',
'^EXTERNAL RISKS$','^DOMESTIC RISKS$','^Weaker-than-expected global growth$','^Potential external risks$','^Potential domestic risks$',
'^Upside risks$','^Downside risks$','^Downside Risks$','^Upside Risks$','^\(B\) Global External Risks$','^Global/External$','^External$','^Internal Risks$',
'^Global$','^Colombia-specific$','^Externally Sourced Risks$','^Domestically Sourced Risks$','^\(A\) Domestic Risks$','^\(B\) External Risks$',
'^Risks to the economic outlook$','^Risks to the financial sector$','^NaN$','^nan$','^Short-term risks$','^Medium-term risks$','^Source of Risks$',
'^Policy and geopolitical uncertainties$','^Policy and geopolitical uncertainties :$','^Financial conditions :$','^Italy$','^Regional / Global$','^Country specific risks$','^Internal \(endogenous\) risks$',
'^Growth slowdown in China:$','^External \(exogenous\) risks$','^Globally-sourced risks$','^Nature\/Source of Threats$',
'^Nature/Source of Main Threats$']
skip_row_pattern = re.compile('|'.join(skip_row), re.I)

In [232]:
def double_extract(df,colname2id,id2colname):
    likelihood_degree_two = '^[Ll]ow\s?/\s?[Mm]edium$|^[Mm]edium\s?/\s?[Ll]ow$|^[Hh]igh\s?/\s?[Mm]edium$|^[Mm]edium\s?/\s?[Hh]igh$|^LOW\s?/\s?MEDIUM$|^MEDIUM\s?/\sLOW$|^HIGH\s?/\s?MEDIUM$|^MEDIUM\s?/\s?HIGH$'
    likelihood_degree_one = '^L$|^[Ll]ow$|^LOW$|^H$|^[hH]igh$|^HIGH$|^M$|^[Mm]edium$|^MEDIUM$'
    likelihood_special = '^MEDIUM to HIGH$|^LOW to MEDIUM$|^S/M$|^M/H$|^Medium to  High$|^Medium-High$|^Low-Medium$|^L/M H/M$|^Medium \(over a 12-month period\)$|^L-M$|^M-H$|^Low/ Medium Medium High/ Medium$|^High / Medium$|^Medium / Low$|^High \(Europe\) Medium \(elsewhere\)$|^High Medium Low / Medium$|^Medium \(Medium-Term\)$|^High \(AEs\) Medium \(EMEs\)$|^Medium to High$|^Low to Medium$|^[Ll]ow\s?/\s?[Mm]edium$|^[Mm]edium\s?/\s?[Ll]ow$|^[Hh]igh\s?/\s?[Mm]edium$|^[Mm]edium\s?/\s?[Hh]igh$|^LOW\s?/\s?MEDIUM$|^MEDIUM\s?/\sLOW$|^HIGH\s?/\s?MEDIUM$|^MEDIUM\s?/\s?HIGH$|^L$|^[Ll]ow$|^LOW$|^H$|^[hH]igh$|^HIGH$|^M$|^[Mm]edium$|^MEDIUM$'    
    for ci in list(colname2id):
            if len(colname2id[ci]) == 0:
                del colname2id[ci]
    
    #if country == 'philippines' and year == 2017:
     #   df = df.drop(6)
      #  df = df.reset_index(drop = True)
    
    for i in colname2id.values():
        for n in range(len(df)):
            if type(df[i[0]][n]) != str:
                df[i[0]][n] ='None'
    if ['risk'] in id2colname.values() and len(colname2id['risk']) != 0:
        if df[colname2id['risk'][0]][0] =='' and df[colname2id['risk'][0]][2] == '':
            for n in range(len(df)):
                if n%2 == 0:
                    df[colname2id['risk'][0]][n] = df[colname2id['risk'][0]][n+1]
    if ['policy'] in id2colname.values() and len(colname2id['policy']) != 0:
        if df[colname2id['policy'][0]][0] =='' and df[colname2id['policy'][0]][2] == '':
            for n in range(len(df)):
                if n%2 == 0:
                    df[colname2id['policy'][0]][n] = df[colname2id['policy'][0]][n+1]
    
    if country == 'czech republic' and year == 2012:
        df=df
    elif ['likelihood'] in id2colname.values() and len(colname2id['likelihood']) != 0:
        if re.search(likelihood_special,df[colname2id['likelihood'][0]][0]) is not None and re.search(likelihood_special,df[colname2id['likelihood'][0]][1]) is None and df[colname2id['likelihood'][0]][1] != '': 
            if df[colname2id['likelihood'][0]][1] != 'None':               
                for n in range(len(df)):
                    if n%2 == 0:
                        df[colname2id['likelihood'][0]][n] = df[colname2id['likelihood'][0]][n] + '&&' + df[colname2id['likelihood'][0]][n+1]
                        df[colname2id['likelihood'][0]][n+1] = df[colname2id['likelihood'][0]][n]
    
    if ['impact'] in id2colname.values() and len(colname2id['impact']) != 0:
        if re.search(likelihood_special,df[colname2id['impact'][0]][0]) is not None and re.search(likelihood_special,df[colname2id['impact'][0]][1]) is None and df[colname2id['impact'][0]][1] != '':
            if country == 'canada' and year == 2018:
                df = df.drop(1)
                df = df.reset_index(drop = True)
            else:
                if df[colname2id['impact'][0]][1] != 'None': 
                    for n in range(len(df)):
                        if n%2 == 0:
                            df[colname2id['impact'][0]][n] = df[colname2id['impact'][0]][n] + '&&' + df[colname2id['impact'][0]][n+1]
                            df[colname2id['impact'][0]][n+1] = df[colname2id['impact'][0]][n]
    

    return df,colname2id,id2colname

In [326]:
report_files_list = glob.glob(dataset_folder+'/*.json')
keywords_record = []
for i, report_file in enumerate(report_files_list):
    with open(report_file, 'r', encoding='utf8') as f:
        data = json.load(f)
        country = data['country']
        year = data['year']
        report_xml = data['xml']
    table_list = extract_risk_table_node_from_report(report_xml)
    if len(table_list) == 0: 
        continue
    contents = defaultdict(list)
    head_name = []
    for table in table_list:
        col_num = get_col_num(table)
        thead_rows = table.findall(".//thead/tr")
        for tr in thead_rows:
            tds = tr.findall(".//th")
            tds.extend(tr.findall(".//td"))
            for td in tds:
                text = ' '.join(td.itertext()).strip()
                head_name.append(text)
        thead_rows_in_tbody = []
        tbody_rows = table.findall(".//tbody/tr")
        if len(tbody_rows) == 0:
            tbody_rows.extend(table.findall(".//tr"))
        
        for i in range(3):
            #check the second and third row if the first row already been pop out
            if len(tbody_rows) >0 and re.search(table_head_pattern, ' '.join(tbody_rows[0].itertext())) is not None:
                thead_rows_in_tbody.append(tbody_rows.pop(0))

        colname2id = defaultdict(list)
        id2colname = defaultdict(list)
        skipid = set()

        colname2id, id2colname, skipid, contents, indicator = \
            new_boundary_case_thead(country, year, colname2id, id2colname, skipid, contents)

        if not indicator:
            colname2id, id2colname, skipid, contents = \
                scan_table_head(colname2id, id2colname, skipid, contents, thead_rows, col_num, \
                            skip_pattern, risk_pattern, likelihood_pattern, impact_pattern, policy_response_pattern, channel_pattern,\
                            direction_pattern, horizon_pattern)
        if country == 'st. kitts and nevis' and year == 2015:
            df = extract_table_special(table_list)
        else:
            df = extract_table(table_list)
            
        if country == 'chad' and year ==2016:
            df=df.drop(0)
            df=df.drop(5)
        elif country =='israel' and year == 2017:
            df=df.drop(0)
        elif country == 'denmark' and year == 2017:
            df[0][6] = 'Medium'
            df[0][4] = 'Medium/High'
        elif country == 'ireland' and year == 2018:
            for n in range(len(df)):
                if n%2 ==0:
                    df[2][n] = df[2][n+1]
            df[0][0] = 'Medium'
            df[0][2] = 'Medium'
            df[0][4] = 'High'
            df[0][6] = 'Medium'
            df[0][8] = 'High'
            df[0][10] = 'Medium'
            df[0][12] = 'Medium'
            df[0][14] = 'Medium'
            df[0][16] = 'Low'      
        df = df.reset_index(drop = True)   
            
        info = country + str(year)
        
        if country == 'togo' and year == 2014:
            df=df.drop(0)
            df=df.drop(1)
            df=df.drop(6)
            df = df.reset_index(drop = True)
            for n in range(len(df)):
                df[2][n] = df[2][n] +'&&' +df[3][n]
            del df[3]
            colname2id['risk'] = [0]
            colname2id['likelihood'] = [1]
            colname2id['impact'] = [2]
            id2colname[0] = ['risk']
            id2colname[1] = ['likelihood']
            id2colname[2] = ['impact']
        elif country == 'saudi arabia' and year == 2012:
            df[0][0]=df[0][0] + '&&' +df[0][1]
            df[0][1]=df[0][0]
            df[1][0]=df[1][0] + '&&' +df[1][1]
            df[1][1]=df[1][0]
            df[2][0]=df[2][0] + '&&' +df[2][1]
            df[2][1]=df[2][0]
        elif country == 'albania' and year == 2014:
            df = albania_2014(df)
        elif country == 'cote divoire' and year == 2018:
            df = df.drop(3)
            df = df.reset_index(drop = True)
        elif country == 'nigeria' and year == 2015:
            df,colname2id,id2colname = nigeria_2015(df,colname2id,id2colname)
        elif country == 'malta' and year == 2018:
            policy = []
            for n in [2,5,8,11,14,17,20]:
                policy.append(df[0][n])
                df = df.drop(n)
            df = df.reset_index(drop = True)
            policy2 = []
            for n in policy:
                policy2.append(n)
                policy2.append(n)
            df[3]=policy2
            colname2id['policy']=[3]
            id2colname[3]=['policy']  
        elif country == 'croatia' and year == 2014:
            df[0][5] = df[0][5] +'&&' +df[0][6]
            df[3][5] = df[3][5] +'&&' +df[3][6]
            df=df.drop(6)
            df = df.reset_index(drop = True)
            df[0][8]=''
        #elif country == 'philippines' and year == 2017:
         #   df,colname2id,id2colname = double_extract(df,colname2id,id2colname)
        elif country == 'comoros' and year == 2015:
            df[0][2] = df[0][1] + '&&'+ df[0][2]
            df[0][4] = df[0][3] +'&&'+ df[0][4]
            df[0][5] = df[0][3] +'&&'+ df[0][5]
            df[0][6] = df[0][3] +'&&'+ df[0][6]
            df[0][8] = df[0][7] +'&&'+ df[0][8]
            for n in [1,3,7]:
                df=df.drop(n)
            df = df.reset_index(drop = True) 
        #elif country == 'mongolia' and year == 2015:
         #   continue 
        #elif country == 'vietnam' and year == 2017:
         #   continue
        elif country == 'kyrgyz' and year == 2013:
            df[0][1] = df[0][1] +'&&'+df[0][2]
            df =df.drop(2)
            df = df.reset_index(drop = True)
            df[0][5] =''
            df[1][1] = ''
            for n in range(3):
                for row in [0,2,4,6,8,10]:
                    df[n][row]=df[n][row] + '&&' + df[n][row + 1]
            for row2 in [1,3,5,7,9,11]:
                df=df.drop(row2)
            df = df.reset_index(drop = True)
        elif country == 'ukraine' and year == 2014:
            df[2][11] = df[2][8] = df[2][9]
            df=df.drop(9)
            df = df.reset_index(drop = True)
        elif country == 'republic of congo' and year == 2014:
            df[2][1] = df[2][0] + '&&' +df[2][1]
            df[2][2] = df[2][0] + '&&' +df[2][2]
            df[2][4] = df[2][3] + '&&' +df[2][4]
            df[2][3] = df[2][4]
            df[2][6] = df[2][5] + '&&' +df[2][6]
            df[2][5] = df[2][6]
            df[2][8] = df[2][7] + '&&' +df[2][8]
            df[2][7] = df[2][8]
            df[1][4] = df[1][3]
            df[1][6] = df[1][5]
            df[1][8] = df[1][7]
            df=df.drop(0)
            df = df.reset_index(drop = True)
        elif country == 'st. kitts and nevis' and year == 2015:
            df=Insert_row(3,df,['',df[0][3],df[1][3],df[2][3],df[3][3]])
            df=Insert_row(5,df,['',df[0][5],df[1][5],df[2][5],df[3][5]])
            df = df.drop(4)
            df=df.drop(6)
            colname2id['risk'].remove(0)
            del id2colname[0]
            df = df.reset_index(drop = True)
        #elif country == 'micronesia' and year == 2017:
         #   continue
        #elif country == 'mongolia' and year == 2017:
         #   continue
        elif country == 'romania' and year == 2016:
            df=df.drop(0)
            df = df.reset_index(drop = True)
        elif country == 'norway' and year == 2015:
            policy = []
            likelihood=[]
            for n in [2,5,8,11,14]:
                policy.append(df[1][n])
                likelihood.append(df[0][n-2])
                df[0][n-2] = ''
                df = df.drop(n)
            df = df.reset_index(drop = True)
            policy2 = []
            likelihood2=[]
            for n in policy:
                policy2.append(n)
                policy2.append(n)
            for n in likelihood:
                likelihood2.append(n)
                likelihood2.append(n)
            df[2]=policy2
            df[3]=likelihood2
            colname2id['likelihood']=[3]
            id2colname[0].remove('likelihood')
            id2colname[3]= ['likelihood']
            colname2id['policy']=[2]
            id2colname[2]=['policy']  
        
        
        else: 
            for n in range(len(id2colname)):
                if id2colname[n] ==['risk','likelihood']:     
                    #df = extract_table(table_list)
                    df,colname2id,id2colname = risk_likelihood(df,colname2id,id2colname)    
                if id2colname[n] ==['impact','policy']:   
                    #df = extract_table(table_list)
                    df,colname2id,id2colname = impact_policy(df,colname2id,id2colname)
                if id2colname[n] ==['risk','likelihood','horizon']:          
                    #df = extract_table(table_list)
                    df,colname2id,id2colname = risk_likelihood_horizon(df,colname2id,id2colname)  
                if id2colname[n] ==['risk','likelihood','direction']:   
                    #df = extract_table(table_list)
                    df,colname2id,id2colname =  risk_likelihood_direction(df,colname2id,id2colname)
                if id2colname[n] ==['risk','likelihood','likelihood']:   
                    #df = extract_table(table_list)
                    df,colname2id,id2colname =  risk_likelihood_likelihood(df,colname2id,id2colname)

            if len(colname2id['risk']) > 1:
                df,colname2id,id2colname = risk_extract(df,colname2id,id2colname)
            if len(colname2id['impact']) > 1:
                df,colname2id,id2colname = impact_extract(df,colname2id,id2colname)
        if [0] in colname2id.values():
            for n in range(len(df)):
                if re.search(skip_row_pattern,str(df[0][n])) is not None:
                    df = df.drop(n)
        df = df.reset_index(drop = True)
        
        if country == 'malta' and year == 2017:
            df,colname2id,id2colname = malta_2017(df,colname2id,id2colname) 
        if country == 'new zealand' and year == 2016:
            df,colname2id,id2colname = new_zealand_2016(df,colname2id,id2colname)
            df = df.reset_index(drop = True)
        if country == 'antigua and barbuda' and year == 2013:
            df,colname2id,id2colname = antigua_and_barbuda(df,colname2id,id2colname)
        if country == 'romania' and year == 2017:
            id2colname[1].remove('channel')
        if country == 'canada' and year == 2016:   
            df=df.drop(7)
            df = df.reset_index(drop = True)
        if country == 'south africa' and year == 2012:
            df[1][4] = df[1][4]+'&&'+df[1][5]
            df[1][5] = df[1][6] + '&&'+df[1][7]
            df=df.drop(6)
            df=df.drop(7)
            df = df.reset_index(drop = True)
        if country == 'azerbaijan' and year==2013:
            df=df.drop(0)
            df=df.drop(5)
            df = df.reset_index(drop = True)
            policy = []
            for n in range(12):
                policy.append(df[0][12])
            df=df.drop(12)
            df[3]=policy
            colname2id['policy'] = [3]
            id2colname[3] = ['policy']
        if country == 'russia' and year==2013:
            df=df.drop(0)
            df = df.reset_index(drop = True)
            df[0][6]=df[0][7]='4.Cyprus'
            colname2id['risk'] = [0]
            colname2id['likelihood'] = [1]
            colname2id['impact'] = [2]
            colname2id['policy'] = [3]
            id2colname[0] = ['risk']
            id2colname[1] = ['likelihood']
            id2colname[2] = ['impact']
            id2colname[3] = ['policy']
        if country == 'fiji' and year == 2018:
            df,colname2id,id2colname = fiji_2018(df,colname2id,id2colname)
        if (country == 'south korea' and year == 2016) or (country=='poland' and year==2014) or (country=='luxembourg' and year==2018) or (country=='greece' and year==2017) or (country=='belgium' and year==2017):
            del id2colname[1]
            id2colname[1] = ['likelihood']
            del colname2id['channel']
        if country == 'botswana' and year == 2013:
            df=df.drop(0)
            df=df.drop(11)
            df = df.reset_index(drop = True)
        if country == 'lesotho' and year == 2014:
            df=df.drop(0)
            df= df.reset_index(drop = True)
        if country == 'republic of congo' and year == 2013:
            for n in range(len(df)):
                if n%2==0:
                    df[2][n+1] = df[2][n]
        if country == 'hong kong' and year == 2018:
            df=df.drop(0)
            df=df.drop(5)
            df = df.reset_index(drop = True)
            df=Insert_row(6,df,['High','High',df[2][6]])
            df=Insert_row(8,df,['High','High',df[2][8]])
            impact =[]
            for n in range(len(df)):
                if n%2==0:
                    impact.append(df[1][n])
                    df=df.drop(n)
            df = df.reset_index(drop = True)
            df[3]=impact
            id2colname[0]= ['likelihood']
            id2colname[1]= ['risk']
            id2colname[2]= ['policy']
            id2colname[3]= ['impact']
            colname2id['likelihood']=[0]
            colname2id['risk']=[1]
            colname2id['policy']=[2]
            colname2id['impact']=[2]   
        if country=='sri lanka' and year == 2014:
            df,colname2id,id2colname=sri_lanka_2014(df,colname2id,id2colname)
        if country=='iran' and year == 2018:
            df,colname2id,id2colname = iran_2018(df,colname2id,id2colname)
        if country=='south korea' and year == 2014:
            df,colname2id,id2colname = south_korea_2014(df,colname2id,id2colname)
        #if country=='south korea' and year == 2014:
         #   df=df.drop(0)
          #  df=df.drop(11)
           # df = df.reset_index(drop = True)
        if country=='montenegro' and year == 2012:
            df=df.drop(0)
            df=df.drop(1)
            df = df.reset_index(drop = True)
        if country=='sweden' and year == 2016:
            df,colname2id,id2colname = sweden_2016(df,colname2id,id2colname)
        if country=='solomon islands' and year == 2018:
            solomon_islands_2018(df,colname2id,id2colname)
        if country=='austria' and year == 2013:
            df[2][9]=df[2][8]+df[2][9]
            df=df.drop(10)
        print(info)
        print(colname2id)
        df,colname2id,id2colname = double_extract(df,colname2id,id2colname)
        print(df)

seychelles2015
defaultdict(<class 'list'>, {'risk': [0], 'likelihood': [1], 'impact': [2], 'policy': [4]})
                                                   0  1  \
0                   Financial distress at major SOEs  M   
1                    Relaxation of fiscal discipline  L   
2  Protracted period of slow growth in advanced a...  H   
3                         Persistent dollar strength  M   
4     Outbreak of disease/perception of health risks  L   
5  Geopolitical fragmentation that erodes the glo...  M   

                                                   2  \
0  M && Either  a falling fiscal balance, a spike...   
1  M && Any relaxation of fiscal discipline in th...   
2  M && If slow growth in Europe and the previous...   
3  M-H && Continued strength of the dollar vs. th...   
4  M-H && Any outbreak of contagious disease in S...   
5  M && If instability in the Middle East were to...   

                                                   4  
0  Improve capacity at PEMC to 

In [ ]:
risk likelihood

In [13]:
#if one key in id2colname conatins risk and likelihood
def risk_likelihood(df,colname2id,id2colname):
    if country == 'st. vincent and the grenadines' and year == 2014:
        df[0][0] = df[0][1]
        df[0][2] = df[0][3]
        df[0][4] = df[0][5]
        df[0][6] = df[0][7]
        df[0][8] = df[0][9]
        df[2][0] = df[2][1]
        df[2][2] = df[2][3]
        df[2][4] = df[2][5]
        df[2][6] = df[2][7]
        df[2][8] = df[2][9]
        likelihood = []
        col = 0 
        for n in range(len(df)):
            if n %2 ==0:
                df[1][n] = df[1][n] + '&&' + df[1][n+1]
                df[1][n+1] = df[1][n]
            if re.search('Likelihood:',df[col][n]) is not None:
                likelihood_location = re.search('Likelihood:',df[col][n]).start()
                content = df[col][n][likelihood_location:]
                likelihood.append(content)
                df[col][n] = df[col][n].replace(content,'')
        df[3] = likelihood
        colname2id['likelihood'] = [3]
        id2colname[0].remove('likelihood')
        id2colname[3] = ['likelihood']
    elif country == 'belgium' and year == 2016:
        df=df.drop(0)
        df = df.reset_index(drop = True)
    elif country == 'israel' and year ==2017:
        likelihood_degree_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
        likelihood_degree_one = ' L |^[Ll]ow$|^LOW$| H |[hH]igh|HIGH| M |^[Mm]edium|MEDIUM'
        likelihood_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
        likelihood_one = '^L |Low|^LOW$|^H |[hH]igh|HIGH|^M |[Mm]edium|MEDIUM'
        df=df.drop(0)
        df=df.drop(7)
        df = df.reset_index(drop = True)
        policy_response=[]
        policy_response.append(df[1][1])
        policy_response.append(df[1][3])
        policy_response.append(df[1][5])
        policy_response.append(df[1][7])
        df=df.drop(1)
        df=df.drop(3)
        df=df.drop(5)
        df=df.drop(7)
        df = df.reset_index(drop = True)
        df[2]=policy_response
        likelihood=[]
        for n in range(len(df)):
            if re.search(likelihood_two,df[0][n]) is not None: 
                content = re.search(likelihood_two,df[0][n],re.I).group()
                likelihood.append(content)
                df[0][n] = df[0][n].replace(content,'')
            elif re.search(likelihood_one,df[0][n]) is not None: 
                content = re.search(likelihood_one,df[0][n],re.I).group()
                likelihood.append(content)
                df[0][n] = df[0][n].replace(content,'')
        df[3] = likelihood
        colname2id['risk'] = [0]
        colname2id['likelihood'] = [3]
        colname2id['impact'] = [1]
        colname2id['policy'] = [2]
        id2colname[0] = ['risk']
        id2colname[3] = ['likelihood']
        id2colname[1] = ['impact']
        id2colname[2] = ['policy']
    else:
        likelihood_degree_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
        likelihood_degree_one = ' L |^[Ll]ow$|^LOW$| H |[hH]igh|HIGH| M |^[Mm]edium|MEDIUM'
        likelihood_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
        likelihood_one = '^L |Low|^LOW$|^H |[hH]igh|HIGH|^M |[Mm]edium|MEDIUM'
        for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood']:
                col = n
        for n in range(len(df)):
            if re.search('Downside Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('Upside Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('Domestic Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('External Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('Internal Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('Externally Sourced Risks',df[0][n]) is not None:
                df = df.drop(n)
            elif re.search('Domestically Sourced Risks',df[0][n]) is not None:
                df = df.drop(n)    
        df = df.reset_index(drop = True)
        
        if country == 'tunisia' and year == 2015:
            df[1][2] = df[1][0]
            df[1][3] = df[1][1]
            row_value = ['High','None']
            df = Insert_row(4, df, row_value) 
            df[1][4] = df[1][0]
            df[1][5] = df[1][1]
            df[0][5] = df[0][5][:len(df[0][5])-5]

        if re.search(likelihood_degree_one,df[0][0]) is None and df[1][0] == None:
            risk_row = []
            risk_content = []
            risk = []
            for n in range(len(df)):
                if df[1][n] == None:
                    risk_row.append(n)
                    risk_content.append(df[0][n])
                    df = df.drop(n)
            df = df.reset_index(drop = True)
            for n in risk_content:
                risk.append(n)
                risk.append(n)
            df[col_num] = risk   
            id2colname[0].remove('risk')
            id2colname[col_num] = ['risk']
            colname2id['risk'] = [col_num]
        
        if re.search(likelihood_degree_two,df[col][0]) is not None or re.search(likelihood_degree_one,df[col][0]) is not None:
            if re.search(likelihood_degree_two,df[col][2]) is not None or re.search(likelihood_degree_one,df[col][2]) is not None:
                if len(df[col][0]) < 15 and len(df[col][2]) <15 :
                    likelihood = []
                    for n in range(len(df)):
                        if n %2 == 0:
                            df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                            df[col][n+1] = df[col][n]
           #             if re.search(likelihood_two,df[col][n]) is not None:
           #                 content = re.search(likelihood_degree_two,df[col][n],re.I).group()
           #                 likelihood.append(content)
           #                 df[col][n] = df[col][n].replace(content,'')
           #             elif re.search(likelihood_one,df[col][n]) is not None:
           #                 content = re.search(likelihood_degree_one,df[col][n],re.I).group()
           #                 likelihood.append(content)
           #                 df[col][n] = df[col][n].replace(content,'')
           #         df[col_num] = likelihood
           #         id2colname[col_num] = ['likelihood']
           #         colname2id['likelihood'] = [col_num]
        
        policy = []
        policy2 = []
        for n in range(len(df)):
            if re.search('Policy response',df[0][n]) is not None:
                policy.append(n)
            if re.search('Policy response',df[1][n]) is not None:
                policy2.append(n)
            elif re.search('Policy Response',df[1][n]) is not None:
                policy2.append(n)
        if len(policy) != 0:
            repeat = []
            repeat.append(policy[0])
            for n in range(len(policy)):
                if n != len(policy) -1 :
                    repeat.append(policy[n+1] - policy[n])
            repeat[0] = repeat[0] + 1
            new_list = []
            for n in range(len(repeat)):
                for i in range(repeat[n]):
                    new_list.append(df[1][policy[n]])
            df[col_num] = new_list
            df = df.drop(policy)
            index = []
            for n in range(len(df)):
                index.append(n)
            df = df.reset_index(drop = True)
            id2colname[col_num ] = ['policy']
            colname2id['policy'] = [col_num ]

            for col in range(len(id2colname)):
                if id2colname[col] == ['risk', 'likelihood']: 
                    likelihood = []
                    for n in range(len(df)-1):
                        if re.search(likelihood_degree_two,df[col][n]) is not None:
                            df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                            #df[col][n+1] = df[col][n] 
                            content = re.search(likelihood_degree_two,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                        elif re.search(likelihood_degree_one,df[col][n]) is not None:
                            content = re.search(likelihood_degree_one,df[col][n],re.I).group()
                            df[col][n] = df[col][n] + '&&'+ df[col][n+1]
                            #df[col][n+1] = df[col][n]
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                            #df[col][n+1] = df[col][n+1].replace(content,'')
                    likelihood_col = []
                    for n in likelihood:
                        likelihood_col.append(n)
                        likelihood_col.append(n)
                    id2colname[col].remove('likelihood')
                    df[col_num + 1] = likelihood_col
                    id2colname[col_num + 1] = ['likelihood']
                    colname2id['likelihood'] = [col_num +1]
                if id2colname[col] == ['impact']: 
                    for n in range(len(df)-1):
                        if re.search(likelihood_degree_two,df[col][n]) is not None:
                            df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                        elif re.search(likelihood_degree_one,df[col][n]) is not None:
                            df[col][n] = df[col][n] + '&&'+ df[col][n+1]

        elif len(policy2) != 0:
            policy_content = []
            for n in policy2:
                policy_content.append(df[1][n])
                policy_content.append(df[1][n])
                if country == 'norway' and year == 2015:
                    policy_content.append(df[1][n])
                df[1][n] = df[1][n-1] 
            df[col_num] = policy_content
            colname2id['policy'] = [col_num]
            id2colname[col_num] = ['policy']

            for n in range(len(id2colname)):
                if id2colname[n] == ['risk', 'likelihood']: 
                    col = n
                    likelihood = []
                    if country == 'norway' and year == 2015:
                        df[0][0] = df[0][0] + '&&' + df[0][1]
                        df[0][1] = df[0][0]
                        df[0][2] = df[0][0]
                        df[0][3] = df[0][3] + '&&' + df[0][4]
                        df[0][4] = df[0][3]
                        df[0][5] = df[0][3]
                        df[0][6] = df[0][6] + '&&' + df[0][7]
                        df[0][7] = df[0][6]
                        df[0][8] = df[0][6]
                        df[0][9] = df[0][9] + '&&' + df[0][10]
                        df[0][10] = df[0][9]
                        df[0][11] = df[0][9]
                        df[0][12] = df[0][12] + '&&' + df[0][13]
                        df[0][13] = df[0][12]
                        df[0][14] = df[0][12]
                        for n in range(len(df)):
                            if re.search(likelihood_two,df[col][n],re.I) is not None:
                                content = re.search(likelihood_two,df[col][n],re.I).group()
                                likelihood.append(content)
                                df[col][n] = df[col][n].replace(content,'')
                            elif re.search(likelihood_one,df[col][n],re.I) is not None:
                                content = re.search(likelihood_one,df[col][n],re.I).group()
                                likelihood.append(content)
                                df[col][n] = df[col][n].replace(content,'')
                        df[col_num +1] =likelihood
                        id2colname[col].remove('likelihood')
                        id2colname[col_num+1] = ['likelihood']
                        colname2id['likelihood'] = [col_num+1]
                    else:
                        for n in range(len(df)):
                            if re.search(likelihood_two,df[col][n],re.I) is not None:
                                content = re.search(likelihood_two,df[col][n],re.I).group()
                                likelihood.append(content)
                                df[col][n] = df[col][n].replace(content,'')
                            elif re.search(likelihood_one,df[col][n],re.I) is not None:
                                content = re.search(likelihood_one,df[col][n],re.I).group()
                                likelihood.append(content)
                                df[col][n] = df[col][n].replace(content,'')
                        df[col_num +1] =likelihood
                        id2colname[col].remove('likelihood')
                        id2colname[col_num+1] = ['likelihood']
                        colname2id['likelihood'] = [col_num+1]       

        else:
            for n in range(len(id2colname)):
                if id2colname[n] == ['risk', 'likelihood']: 
                    col = n
                    likelihood = []
                    for n in range(len(df)):
                        if re.search(likelihood_two,df[col][n]) is not None:
                            content = re.search(likelihood_two,df[col][n]).group()
                            df[col][n] = df[col][n].replace(content,'')
                            likelihood.append(content)
                        elif re.search(likelihood_one,df[col][n]) is not None:
                            content = re.search(likelihood_one,df[col][n]).group()
                            df[col][n] = df[col][n].replace(content,'')
                            likelihood.append(content)
                    df[col_num] =likelihood
                    id2colname[col].remove('likelihood')
                    id2colname[col_num] = ['likelihood']
                    colname2id['likelihood'] = [col_num]
    return df,colname2id,id2colname

In [ ]:
impact policy

In [14]:
#if one key in id2colname conatins impact and policy
def impact_policy(df,colname2id,id2colname):
    likelihood_degree_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_degree_one = ' L |^[Ll]ow$|^LOW$| H |[hH]igh|HIGH| M |^[Mm]edium|MEDIUM'
    likelihood_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_one = '^L |Low|^LOW$|^H |[hH]igh|HIGH|^M |[Mm]edium|MEDIUM'
    for n in range(len(id2colname)):
        if id2colname[n] == ['impact', 'policy']:
            col = n
    for n in range(len(df)):
        if re.search('Potential Domestic Shocks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Potential External Shocks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Internal Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Global risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('(A) Domestic Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('(B) External Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Risks to the economic outlook',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Risks to the financial sector',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Policy responses to shocks',df[0][n]) is not None:
            policy_content = df[0][n]
            policy_special = []
            df = df.drop(n)  
            for n in range(len(df)):
                policy_special.append(policy_content)
            df[col_num] = policy_special
            id2colname[col].remove('policy')
            id2colname[col_num] = ['policy']
            colname2id['policy'] = [col_num]
    df = df.reset_index(drop = True)
 
   # if country == 'tunisia' and year == 2015:
   #     df[1][2] = df[1][0]
   #     df[1][3] = df[1][1]
   #     row_value = ['High','None']
   #     df = Insert_row(4, df, row_value) 
   #     df[1][4] = df[1][0]
   #     df[1][5] = df[1][1]
   #     df[0][5] = df[0][5][:len(df[0][5])-5]
    if (country == 'botswana' and year ==2016) or (country == 'myanmar' and year ==2014):
        row_value = [df[0][10],'None','None']
        df = Insert_row(12, df, row_value)
        for n in range(len(df)):
            if n%2 == 1:
                df[2][n-1] = df[2][n]
                

    if len(df[col][0]) >15:
        policy = []
        impact = []
        if country =='maldives' and year == 2015:
            if re.search('Policy actions',df[2][0]) is not None:
                policy = []
                policy_location = re.search('Policy actions',df[2][0]).start()
                content = df[2][0][policy_location:]
                for n in range(len(df)):
                    policy.append(content)
                df[2][0] = df[2][0].replace(content,'')
        elif country == 'japan' and year ==2012:
            policy.append(df[2][1])
            policy.append(df[2][1])
            df = df.drop(1)
            df = df.reset_index(drop = True)
            for n in range(len(df)):
                if re.search('Policy response',df[2][n]) is not None:
                    policy_location = re.search('Policy response',df[2][n]).start()
                    content = df[2][n][policy_location:]
                    policy.append(content)
                    df[2][n] = df[2][n].replace(content,'')
            df[2][1] = df[2][0]
        
        else:
            for n in range(len(df)):
                if re.search('Policy recommendation',df[col][n]) is not None:
                    policy_location = re.search('Policy recommendation',df[col][n]).start()
                    content = df[col][n][policy_location:]
                    policy.append(content)
                    df[col][n] = df[col][n].replace(content,'')
                elif re.search('Policy response',df[col][n]) is not None:
                    policy_location = re.search('Policy response',df[col][n]).start()
                    content = df[col][n][policy_location:]
                    policy.append(content)
                    df[col][n] = df[col][n].replace(content,'')
                elif re.search('Policy Response',df[col][n]) is not None:
                    policy_location = re.search('Policy Response',df[col][n]).start()
                    content = df[col][n][policy_location:]
                    policy.append(content)
                    df[col][n] = df[col][n].replace(content,'')
            if country == 'china' and year ==2014:
                policy.append(df[3][4])
                policy.append(df[3][5])
            elif country == 'tunisia' and year ==2012:
                df =df.drop(1)
                df =df.drop(3)
                df =df.drop(5)
                df =df.drop(7)
                df =df.drop(9)
                df =df.drop(11)
                df = df.reset_index(drop = True)
            elif country == 'thailand' and year == 2013:
                policy.append('None')
            
      #      elif re.search(likelihood_two,df[col][n]) is not None:
      #          content = re.search(likelihood_two,df[col][n],re.I).group()
      #          impact.append(content)
      #          df[col][n] = df[col][n].replace(content,'')
      #      elif re.search(likelihood_one,df[col][n]) is not None:
      #          content = re.search(likelihood_one,df[col][n],re.I).group()
      #          impact.append(content)
      #          df[col][n] = df[col][n].replace(content,'')
        if len(policy) != 0 :
            df[col_num] = policy
            id2colname[col].remove('policy')
            id2colname[col_num] = ['policy']
            colname2id['policy'] = [col_num]
      #  elif len(impact) != 0:
      #      df[col_num] = impact
      #      id2colname[col].remove('impact')
      #      id2colname[col_num] = ['impact']
      #      colname2id['impact'] = [col_num]
    else:
        impact = []
        if (country == 'kiribati' and year ==2016) or (country == 'tunisia' and year ==2018):
            df[1][1] = df[1][0] + '&&' +df[1][1]
            impact.append(df[1][1])
            impact.append(df[1][1])
            impact.append(df[1][1])
            impact.append(df[1][1])
            df[1][5] = df[1][4] + '&&' +df[1][5]
            impact.append(df[1][5])
            impact.append(df[1][5])
            df[1][7] = df[1][6] + '&&' +df[1][7]
            impact.append(df[1][7])
            impact.append(df[1][7])
            df[1][9] = df[1][8] + '&&' +df[1][9]
            impact.append(df[1][9])
            impact.append(df[1][9])
            df[1] = impact
        else:
            for n in range(len(df)):
                if n%2 == 0:
                    df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                    df[col][n+1] = df[col][n] 
            for n in range(len(df)):
                if re.search(likelihood_two,df[col][n]) is not None:
                    content = re.search(likelihood_two,df[col][n],re.I).group()
                    impact.append(content)
                    df[col][n] = df[col][n].replace(content,'')
                elif re.search(likelihood_one,df[col][n]) is not None:
                    content = re.search(likelihood_one,df[col][n],re.I).group()
                    impact.append(content)
                    df[col][n] = df[col][n].replace(content,'')
        df[col_num] = impact
        id2colname[col].remove('impact')
        id2colname[col_num] = ['impact']
        colname2id['impact'] = [col_num]
    return df,colname2id,id2colname

In [ ]:
risk likelihood horizon

In [15]:
#if one key in id2colname conatins risk and likelihood and horizon
def risk_likelihood_horizon(df,colname2id,id2colname):
    likelihood_degree_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_degree_one = ' L |Low |^LOW$| H |High |HIGH| M |Medium |MEDIUM'
    likelihood_two = '[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_one = ' L |Low|^LOW$| H |High|HIGH| M |Medium|MEDIUM'

    for n in range(len(df)):
        if re.search('Global',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Colombia-specific',df[0][n]) is not None:
            df = df.drop(n)
    df = df.reset_index(drop = True)
    for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','horizon']:
                col = n
    for n in range(len(df)):
        if re.search('Downside Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Upside Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Internal Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Global',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Colombia-specific',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Externally Sourced Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestically Sourced Risks',df[0][n]) is not None:
            df = df.drop(n)    
    df = df.reset_index(drop = True)
     
    if re.search(likelihood_degree_one,df[0][0]) is None and df[1][0] == None:
        risk_row = []
        risk_content = []
        risk = []
        for n in range(len(df)):
            if df[1][n] == None:
                risk_row.append(n)
                risk_content.append(df[0][n])
                df = df.drop(n)
        df = df.reset_index(drop = True)
        for n in risk_content:
            risk.append(n)
            risk.append(n)
        df[col_num] = risk   
        id2colname[0].remove('risk')
        id2colname[col_num] = ['risk']
        colname2id['risk'] = [col_num]
        
    if re.search(likelihood_degree_two,df[col][0]) is not None or re.search(likelihood_degree_one,df[col][0]) is not None:
        if re.search(likelihood_degree_two,df[col][2]) is not None or re.search(likelihood_degree_one,df[col][2]) is not None:
             if len(df[col][0]) < 15 and len(df[col][2]) <15 :
                likelihood = []
                for n in range(len(df)):
                    if n %2 == 0:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                        df[col][n+1] = df[col][n]
            
    policy = []
    policy2 = []
    for n in range(len(df)):
        if re.search('Policy response',df[0][n]) is not None:
            policy.append(n)
        if re.search('Policy response',df[1][n]) is not None:
            policy2.append(n)
        elif re.search('Policy Response',df[1][n]) is not None:
            policy2.append(n)
    if len(policy) != 0:
        repeat = []
        repeat.append(policy[0])
        for n in range(len(policy)):
            if n != len(policy) -1 :
                repeat.append(policy[n+1] - policy[n])
        repeat[0] = repeat[0] + 1
        new_list = []
        for n in range(len(repeat)):
            for i in range(repeat[n]):
                new_list.append(df[1][policy[n]])
        df[col_num] = new_list
        df = df.drop(policy)
        index = []
        for n in range(len(df)):
            index.append(n)
        df = df.reset_index(drop = True)
        id2colname[col_num ] = ['policy']
        colname2id['policy'] = [col_num ]

        for col in range(len(id2colname)):
            if id2colname[col] == ['risk', 'likelihood','horizon']: 
                likelihood = []
                for n in range(len(df)-1):
                    if re.search(likelihood_degree_two,df[col][n]) is not None:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                        #df[col][n+1] = df[col][n] 
                        content = re.search(likelihood_degree_two,df[col][n],re.I).group()
                        likelihood.append(content)
                        df[col][n] = df[col][n].replace(content,'')
                    elif re.search(likelihood_degree_one,df[col][n]) is not None:
                        content = re.search(likelihood_degree_one,df[col][n],re.I).group()
                        df[col][n] = df[col][n] + '&&'+ df[col][n+1]
                        #df[col][n+1] = df[col][n]
                        likelihood.append(content)
                        df[col][n] = df[col][n].replace(content,'')
                        #df[col][n+1] = df[col][n+1].replace(content,'')
                likelihood_col = []
                for n in likelihood:
                    likelihood_col.append(n)
                    likelihood_col.append(n)
                id2colname[col].remove('likelihood')
                df[col_num + 1] = likelihood_col
                id2colname[col_num + 1] = ['likelihood']
                colname2id['likelihood'] = [col_num +1]
            if id2colname[col] == ['impact']: 
                for n in range(len(df)-1):
                    if re.search(likelihood_degree_two,df[col][n]) is not None:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                    elif re.search(likelihood_degree_one,df[col][n]) is not None:
                        df[col][n] = df[col][n] + '&&'+ df[col][n+1]
    elif len(policy2) != 0:
        policy_content = []
        for n in policy2:
            policy_content.append(df[1][n])
            policy_content.append(df[1][n])
            if country == 'norway' and year == 2015:
                policy_content.append(df[1][n])
            df[1][n] = df[1][n-1] 
        df[col_num] = policy_content
        colname2id['policy'] = [col_num]
        id2colname[col_num] = ['policy']

        for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','horizon']: 
                col = n
                likelihood = []
                if country == 'norway' and year == 2015:
                    df[0][0] = df[0][0] + '&&' + df[0][1]
                    df[0][1] = df[0][0]
                    df[0][2] = df[0][0]
                    df[0][3] = df[0][3] + '&&' + df[0][4]
                    df[0][4] = df[0][3]
                    df[0][5] = df[0][3]
                    df[0][6] = df[0][6] + '&&' + df[0][7]
                    df[0][7] = df[0][6]
                    df[0][8] = df[0][6]
                    df[0][9] = df[0][9] + '&&' + df[0][10]
                    df[0][10] = df[0][9]
                    df[0][11] = df[0][9]
                    df[0][12] = df[0][12] + '&&' + df[0][13]
                    df[0][13] = df[0][12]
                    df[0][14] = df[0][12]
                    for n in range(len(df)):
                        if re.search(likelihood_two,df[col][n],re.I) is not None:
                            content = re.search(likelihood_two,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                        elif re.search(likelihood_one,df[col][n],re.I) is not None:
                            content = re.search(likelihood_one,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                    df[col_num +1] =likelihood
                    id2colname[col].remove('likelihood')
                    id2colname[col_num+1] = ['likelihood']
                    colname2id['likelihood'] = [col_num+1]
                else:
                    for n in range(len(df)):
                        if re.search(likelihood_two,df[col][n],re.I) is not None:
                            content = re.search(likelihood_two,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                        elif re.search(likelihood_one,df[col][n],re.I) is not None:
                            content = re.search(likelihood_one,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                    df[col_num +1] =likelihood
                    id2colname[col].remove('likelihood')
                    id2colname[col_num+1] = ['likelihood']
                    colname2id['likelihood'] = [col_num+1]
                        
    else:
        for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','horizon']: 
                col = n
                likelihood = []
                for n in range(len(df)):
                    if re.search(likelihood_two,df[col][n]) is not None:
                        content = re.search(likelihood_two,df[col][n]).group()
                        df[col][n] = df[col][n].replace(content,'')
                    elif re.search(likelihood_one,df[col][n]) is not None:
                        content = re.search(likelihood_one,df[col][n]).group()
                        df[col][n] = df[col][n].replace(content,'')
                    likelihood.append(content)
                df[col_num] =likelihood
                id2colname[col].remove('likelihood')
                id2colname[col_num] = ['likelihood']
                colname2id['likelihood'] = [col_num]
    return df,colname2id,id2colname

In [ ]:
risk likelihood direction

In [16]:
#if one key in id2colname conatins risk and likelihood and direction
def risk_likelihood_direction(df,colname2id,id2colname):
    likelihood_degree_two = '[Ll]ow  to  [Mm]edium|[Mm]edium  to  [Hh]igh|[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_degree_one = ' L |Low |^LOW$| H |High |HIGH| M |Medium |MEDIUM'
    likelihood_two = '[Ll]ow  to  [Mm]edium|[Mm]edium  to  [Hh]igh|[Ll]ow\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Ll]ow|[Hh]igh\s?/\s?[Mm]edium|[Mm]edium\s?/\s?[Hh]igh|LOW\s?/\s?MEDIUM|MEDIUM\s?/\sLOW|HIGH\s?/\s?MEDIUM|MEDIUM\s?/\s?HIGH *'
    likelihood_one = ' L |Low|^LOW$| H |High|HIGH| M |Medium|MEDIUM'

   
    for n in range(len(df)):
        if re.search('Global',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Colombia-specific',df[0][n]) is not None:
            df = df.drop(n)
    df = df.reset_index(drop = True)
    for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','direction']:
                col = n
    for n in range(len(df)):
        if re.search('Downside Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Upside Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestic',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Global/External',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('External',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Internal Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Global',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Colombia-specific',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Externally Sourced Risks',df[0][n]) is not None:
            df = df.drop(n)
        elif re.search('Domestically Sourced Risks',df[0][n]) is not None:
            df = df.drop(n)    
    df = df.reset_index(drop = True)
      
    if re.search(likelihood_degree_one,df[0][0]) is None and df[1][0] == None:
        risk_row = []
        risk_content = []
        risk = []
        for n in range(len(df)):
            if df[1][n] == None:
                risk_row.append(n)
                risk_content.append(df[0][n])
                df = df.drop(n)
        df = df.reset_index(drop = True)
        for n in risk_content:
            risk.append(n)
            risk.append(n)
        df[col_num] = risk   
        id2colname[0].remove('risk')
        id2colname[col_num] = ['risk']
        colname2id['risk'] = [col_num]
        
    if re.search(likelihood_degree_two,df[col][0]) is not None or re.search(likelihood_degree_one,df[col][0]) is not None:
        if re.search(likelihood_degree_two,df[col][2]) is not None or re.search(likelihood_degree_one,df[col][2]) is not None:
             if len(df[col][0]) < 15 and len(df[col][2]) <15 :
                likelihood = []
                for n in range(len(df)):
                    if n %2 == 0:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                        df[col][n+1] = df[col][n]
            
    policy = []
    policy2 = []
    for n in range(len(df)):
        if re.search('Policy response',df[0][n]) is not None:
            policy.append(n)
        if re.search('Policy response',df[1][n]) is not None:
            policy2.append(n)
        elif re.search('Policy Response',df[1][n]) is not None:
            policy2.append(n)
    if len(policy) != 0:
        repeat = []
        repeat.append(policy[0])
        for n in range(len(policy)):
            if n != len(policy) -1 :
                repeat.append(policy[n+1] - policy[n])
        repeat[0] = repeat[0] + 1
        new_list = []
        for n in range(len(repeat)):
            for i in range(repeat[n]):
                new_list.append(df[1][policy[n]])
        df[col_num] = new_list
        df = df.drop(policy)
        index = []
        for n in range(len(df)):
            index.append(n)
        df = df.reset_index(drop = True)
        id2colname[col_num ] = ['policy']
        colname2id['policy'] = [col_num ]

        for col in range(len(id2colname)):
            if id2colname[col] == ['risk', 'likelihood','direction']: 
                likelihood = []
                for n in range(len(df)-1):
                    if re.search(likelihood_degree_two,df[col][n]) is not None:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                        #df[col][n+1] = df[col][n] 
                        content = re.search(likelihood_degree_two,df[col][n],re.I).group()
                        likelihood.append(content)
                        df[col][n] = df[col][n].replace(content,'')
                    elif re.search(likelihood_degree_one,df[col][n]) is not None:
                        content = re.search(likelihood_degree_one,df[col][n],re.I).group()
                        df[col][n] = df[col][n] + '&&'+ df[col][n+1]
                        #df[col][n+1] = df[col][n]
                        likelihood.append(content)
                        df[col][n] = df[col][n].replace(content,'')
                        #df[col][n+1] = df[col][n+1].replace(content,'')
                likelihood_col = []
                for n in likelihood:
                    likelihood_col.append(n)
                    likelihood_col.append(n)
                id2colname[col].remove('likelihood')
                df[col_num + 1] = likelihood_col
                id2colname[col_num + 1] = ['likelihood']
                colname2id['likelihood'] = [col_num +1]
            if id2colname[col] == ['impact']: 
                for n in range(len(df)-1):
                    if re.search(likelihood_degree_two,df[col][n]) is not None:
                        df[col][n] = df[col][n] +'&&'+ df[col][n+1]
                    elif re.search(likelihood_degree_one,df[col][n]) is not None:
                        df[col][n] = df[col][n] + '&&'+ df[col][n+1]
    elif len(policy2) != 0:
        policy_content = []
        for n in policy2:
            policy_content.append(df[1][n])
            policy_content.append(df[1][n])
            if country == 'norway' and year == 2015:
                policy_content.append(df[1][n])
            df[1][n] = df[1][n-1] 
        df[col_num] = policy_content
        colname2id['policy'] = [col_num]
        id2colname[col_num] = ['policy']

        for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','direction']: 
                col = n
                likelihood = []
                if country == 'norway' and year == 2015:
                    df[0][0] = df[0][0] + '&&' + df[0][1]
                    df[0][1] = df[0][0]
                    df[0][2] = df[0][0]
                    df[0][3] = df[0][3] + '&&' + df[0][4]
                    df[0][4] = df[0][3]
                    df[0][5] = df[0][3]
                    df[0][6] = df[0][6] + '&&' + df[0][7]
                    df[0][7] = df[0][6]
                    df[0][8] = df[0][6]
                    df[0][9] = df[0][9] + '&&' + df[0][10]
                    df[0][10] = df[0][9]
                    df[0][11] = df[0][9]
                    df[0][12] = df[0][12] + '&&' + df[0][13]
                    df[0][13] = df[0][12]
                    df[0][14] = df[0][12]
                    for n in range(len(df)):
                        if re.search(likelihood_two,df[col][n],re.I) is not None:
                            content = re.search(likelihood_two,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                        elif re.search(likelihood_one,df[col][n],re.I) is not None:
                            content = re.search(likelihood_one,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                    df[col_num +1] =likelihood
                    id2colname[col].remove('likelihood')
                    id2colname[col_num+1] = ['likelihood']
                    colname2id['likelihood'] = [col_num+1]
                else:
                    for n in range(len(df)):
                        if re.search(likelihood_two,df[col][n],re.I) is not None:
                            content = re.search(likelihood_two,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                        elif re.search(likelihood_one,df[col][n],re.I) is not None:
                            content = re.search(likelihood_one,df[col][n],re.I).group()
                            likelihood.append(content)
                            df[col][n] = df[col][n].replace(content,'')
                    df[col_num +1] =likelihood
                    id2colname[col].remove('likelihood')
                    id2colname[col_num+1] = ['likelihood']
                    colname2id['likelihood'] = [col_num+1]
                        
    else:
        for n in range(len(id2colname)):
            if id2colname[n] == ['risk', 'likelihood','direction']: 
                col = n
                likelihood = []
                for n in range(len(df)):
                    if re.search(likelihood_two,df[col][n]) is not None:
                        content = re.search(likelihood_two,df[col][n]).group()
                        df[col][n] = df[col][n].replace(content,'')
                    elif re.search(likelihood_one,df[col][n]) is not None:
                        content = re.search(likelihood_one,df[col][n]).group()
                        df[col][n] = df[col][n].replace(content,'')
                    likelihood.append(content)
                df[col_num] =likelihood
                id2colname[col].remove('likelihood')
                id2colname[col_num] = ['likelihood']
                colname2id['likelihood'] = [col_num]
    return df,colname2id,id2colname

In [ ]:
risk likelihood likelihood

In [17]:
#if one key in id2colname conatins risk and likelihood and likeoihood
def risk_likelihood_likelihood(df,colname2id,id2colname):
    if country == 'bangladesh' and year == 2017:
        df[1][0] = df[1][1]
        df[1][2] = df[1][3]
        df[1][4] = df[1][5]
        df[1][6] = df[1][7]
        df[1][8] = df[1][9]
        df[1][10] = df[1][11]
        df[2][0] = df[2][1]
        df[2][2] = df[2][3]
        df[2][4] = df[2][5]
        df[2][6] = df[2][7]
        df[2][8] = df[2][9]
        df[2][10] = df[2][11]
        risk = ['Low to Medium','Low to Medium', 'Medium','Medium','Medium','Medium','High','High','High','High','High','High']
        impact = ['High','High','Medium','Medium','Medium to High','Medium to High','Medium to High','Medium to High','Medium','Medium','Medium','Medium']
        policy = ['Fiscal, Monetary/exchange rate','Fiscal,Financial','Fiscal','Fiscal, Monetary/exchange rate','Fiscal, Monetary/exchange rate','Fiscal, Monetary/exchange rate']
        list = [0,2,4,6,8,10] 
        for n in range(len(list)):
            df[3][list[n]] = policy[n] +'&&' +df[3][list[n]+1]
            df[3][list[n]+1] = df[3][list[n]]
        df[4] = risk
        for n in range(len(df)):
            df[2][n] = impact[n] + '&&' + df[2][n]
        del df[0]
        del df[5]
        del df[6]

        id2colname[1].remove('likelihood')
        id2colname[1].remove('likelihood')
        id2colname[4] = ['likelihood']
        del id2colname[5]
        del id2colname[6]
        colname2id['likelihood'] = [4]
        colname2id['policy'] = [3]
    else:
        likelihood = ['Low to Medium','Medium','Medium','Medium to High','High','High']
        impact = ['High','Medium','Medium to High','Medium','Medium','Medium']
        policy = ['Fiscal, Monetary/exchange rate','Fiscal,Financial','Fiscal','Fiscal, Monetary/exchange rate','Monetary/exchange rate','Fiscal, Monetary/exchange rate']
        df[4] = likelihood
        for n in range(len(df)):
            df[2][n] = impact[n] + '&&' + df[2][n]
        del df[0]
        
        id2colname[1].remove('likelihood')
        id2colname[1].remove('likelihood')
        id2colname[4] = ['likelihood']
        del id2colname[0]
        del id2colname[5]
        del id2colname[6]
        colname2id['risk'] = [1]
        colname2id['policy'] = [3]
        colname2id['likelihood'] = [4]
    
    return df,colname2id,id2colname

In [18]:
#if colname2id['impact'] contains more than one colomns
def impact_extract(df,colname2id,id2colname):
    if len(colname2id['impact']) > 1:
        # print(info)
        #if country == 'st. kitts and nevis' and year == 2015:
        #    df = extract_table_special(table_list)
        #else:
        #    df = extract_table(table_list)
        #print(extract_table(table_list))
        
     #   empty_column = [r'^global',r'^domestic( risks*)*',r'^DOMESTC RISKS',r'^external( risks*)*',r'^external( risks*)*',r'^country(?:[- ])specific(?:[- ])risks*',\
     #          r'^short(?: )*/(?: )*medium terms*',r'^medium(?:[- ])terms*',r'^short(?:[- ])terms*',r'^short(?:[- ]term risks*)',r'^medium(?:[- ]term risks*)',r'^regional risks*',r'^short terms*',r'^global risks',]
     #   empty_column_pattern = re.compile('|'.join(empty_column), re.I)
        
        if country == 'marshall islands' and year==2016:
            df = df.drop(0)
            df = df.drop(5)
        elif country == 'palau' and year==2016:
            df = df
        else:
            if [0] in colname2id.values():
                for n in range(len(df)):
                    if re.search(skip_row_pattern,df[0][n]) is not None:
                        df = df.drop(n)
        df = df.reset_index(drop = True)
        
        
     #   n = colname2id['impact'][0]
        #print(info)
     #   if re.search(empty_column_pattern,df[n][1]) is not None or re.search(empty_column_pattern,df[n][1]) is not None:
     #       del df[n]
     #       colname2id['impact'].remove(n)
     #   elif re.search(empty_column_pattern,df[n+1][1]) is not None or re.search(empty_column_pattern,df[n+1][1]) is not None:
     #       del df[n+1]
     #       colname2id['impact'].remove(n+1)
           
        if 'Impact' in head_name and 'Effect' in head_name:
            num1 = head_name.index('Impact')
            num2 = head_name.index('Effect')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['impact'].remove(num2)
        elif 'Severity' in head_name and 'Impact' in head_name:
            num1 = head_name.index('Severity')
            num2 = head_name.index('Impact')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['impact'].remove(num2)
        elif 'Expected impact on the economy if risk is realized' in head_name and 'Relative Impact' in head_name:
            num2 = head_name.index('Expected impact on the economy if risk is realized')
            num1 = head_name.index('Relative Impact')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['impact'].remove(num2)
        elif 'Impact if Realized' in head_name and 'Potential Impact' in head_name:
            if len(head_name) < col_num:
                num1 = head_name.index('Impact if Realized') + 1 
                num2 = head_name.index('Potential Impact') + 1
                df[num1] = df[num1] +' && ' +df[num2]
                del df[num2]
                colname2id['impact'].remove(num2)
            else:
                num1 = head_name.index('Impact if Realized') 
                num2 = head_name.index('Potential Impact') 
                df[num1] = df[num1] +' && ' +df[num2]
                del df[num2]
                colname2id['impact'].remove(num2)
        elif 'Impact if realized' in head_name and 'Potential impact' in head_name:
            num1 = head_name.index('Impact if realized') 
            num2 = head_name.index('Potential impact') 
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['impact'].remove(num2)
        elif 'Potential Impact' in head_name and 'Nature of vulnerabilities and expected impact if risk materializes' in head_name:
        #print(colname2id['impact'])
            num1 = head_name.index('Potential Impact') 
            num2 = head_name.index('Nature of vulnerabilities and expected impact if risk materializes') 
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
        #print(colname2id['impact'])
            colname2id['impact'].remove(num2)
        else:
            impact=[]
            for n in range(len(df)):
                df[colname2id['impact'][0]][n] = df[colname2id['impact'][0]][n] +'&&'+ df[colname2id['impact'][1]][n]
            del df[colname2id['impact'][1]]
            colname2id['impact'].remove(colname2id['impact'][1])
    return df,colname2id,id2colname

In [28]:
#if colname2id['risk'] contains more than one colomns
def risk_extract(df,colname2id,id2colname):
    empty_column = [r'global',r'domestic( risks*)*',r'DOMESTC RISKS',r'external( risks*)*',r'external( risks*)*',r'country(?:[- ])specific(?:[- ])risks*',\
            r'short(?: )*/(?: )*medium terms*',r'medium(?:[- ])terms*',r'short(?:[- ])terms*',r'short(?:[- ]term risks*)',r'medium(?:[- ]term risks*)',r'regional risks*',r'short terms*',r'global risks',]
    empty_column_pattern = re.compile('|'.join(empty_column), re.I)
            
    if len(colname2id['risk']) > 1:
       # print(info)
       # if country == 'st. kitts and nevis' and year == 2015:
       #     df = extract_table_special(table_list)
       # else:
       #     df = extract_table(table_list)
       # if country == 'algeria' and year == 2016:
       #     continue
        #print(extract_table(table_list))
        if re.search(empty_column_pattern,df[0][2]) is not None and re.search(empty_column_pattern,df[0][len(df)-2]) is not None:
            del df[0]
            colname2id['risk'].remove(0)
        elif re.search(empty_column_pattern,df[0][0]) is not None and df[0][1] == '':
            del df[0]
            colname2id['risk'].remove(0)
        elif df[0][1] =='1.':
            del df[0]
            colname2id['risk'].remove(0)
        
        if country == 'algeria' and year==2014:
            df,colname2id,id2colname = algeria_2014(df,colname2id,id2colname)
        elif country == 'hungary' and year == 2015:
            df,colname2id,id2colname =hungary_2015(df,colname2id,id2colname)
        elif country == 'hungary' and year == 2014:
            df,colname2id,id2colname =hungary_2014(df,colname2id,id2colname)
        elif country == 'colombia' and year == 2014:
            df,colname2id,id2colname =colombia_2014(df,colname2id,id2colname)
        elif country == 'algeria' and year == 2017:
            df,colname2id,id2colname =algeria_2017(df,colname2id,id2colname)
        elif country == 'algeria' and year == 2016:
            df,colname2id,id2colname =algeria_2016(df,colname2id,id2colname)
        
        elif 'Source of Risk' in head_name and 'Nature of Impact' in head_name:
            num1 = head_name.index('Source of Risk')
            num2 = head_name.index('Nature of Impact')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['risk'].remove(num2)
        elif 'Shock' in head_name and 'Vulnerability' in head_name:
            num1 = head_name.index('Shock')
            num2 = head_name.index('Vulnerability')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['risk'].remove(num2)
        elif 'Shock' in head_name and 'Vulnerabilities' in head_name:
            num1 = head_name.index('Shock')
            num2 = head_name.index('Vulnerabilities')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['risk'].remove(num2)
        elif 'Risk' in head_name and 'Description' in head_name:
            num1 = head_name.index('Risk')
            num2 = head_name.index('Description')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['risk'].remove(num2)
        elif 'Source of Risks' in head_name and 'Source of impact' in head_name:
            num1 = head_name.index('Source of Risks')
            num2 = head_name.index('Source of impact')
            df[num1] = df[num1] +' && ' +df[num2]
            del df[num2]
            colname2id['risk'].remove(num2) 
        elif 'Nature/Source of Main Threats' in head_name and 'Risk' in head_name:
            num= head_name.index('Risk')
            colname2id['risk'].remove(num)
            colname2id['likelihood'] = [num]
        elif 'Source of risk' in head_name and 'Risk' in head_name:
            num= head_name.index('Risk')
            colname2id['risk'].remove(num)
            colname2id['likelihood'] = [num]
        elif 'Sources of risk' in head_name and 'Risk' in head_name:
            num= head_name.index('Risk')
            colname2id['risk'].remove(num)
            colname2id['likelihood'] = [num]
        elif 'Source of Risk' in head_name and 'Risk' in head_name:
            num= head_name.index('Risk')
            colname2id['risk'].remove(num)
            colname2id['likelihood'] = [num]
        elif 'Source of Risk' in head_name and 'Risk 2' in head_name:
            num= head_name.index('Risk 2')
            colname2id['risk'].remove(num)
            colname2id['likelihood'] = [num]
        
    return df,colname2id,id2colname

In [ ]:
Special cases

In [20]:
def nigeria_2015(df,colname2id,id2colname):
    for n in [0,20,30,43,51]:
        df=df.drop(n)
    df = df.reset_index(drop = True)
    for n in range(5):
        df[n][0] = df[n][0] +' '+df[n][1] +' '+df[n][2]
        df[n][1]=df[n][2]=''
        df[n][3] = df[n][3] +' '+df[n][4] 
        df[n][4] = ''
        df[n][5] = df[n][5] +' '+df[n][6] +' '+df[n][7] +' '+df[n][8] +' '+df[n][9] 
        df[n][6] = df[n][7] =df[n][8] =df[n][9] =''
        df[n][10] = df[n][10] +' '+df[n][11] 
        df[n][11] = ''
        df[n][12] = df[n][12] +' '+df[n][13] +' '+df[n][14]
        df[n][13]=df[n][14]=''
        df[n][19] = df[n][19] +' '+df[n][20] +' '+df[n][21] +' '+df[n][22] +' '+df[n][23] +' '+df[n][24] +' '+df[n][25]+' '+df[n][26]+' '+df[n][27]
        df[n][20] = df[n][21] =df[n][22] =df[n][23] =df[n][24]=df[n][25]=df[n][26]=df[n][27]=''
        df[n][28] = df[n][28] +' '+df[n][29] +' '+df[n][30] +' '+df[n][31] +' '+df[n][32] +' '+df[n][33] +' '+df[n][34]+' '+df[n][35]
        df[n][29] = df[n][30] =df[n][31] =df[n][32] =df[n][33]=df[n][34]=df[n][35]=''
        df[n][36] = df[n][36] +' '+df[n][37] +' '+df[n][38] +' '+df[n][39]
        df[n][37] = df[n][38] =df[n][39] =''
        df[n][40] = df[n][40] +' '+df[n][41] +' '+df[n][42] +' '+df[n][43] +' '+df[n][44] +' '+df[n][45] 
        df[n][41] = df[n][42] =df[n][43] =df[n][44] =df[n][45]=''
        df[n][47] = df[n][47] +' '+df[n][48] +' '+df[n][49]
        df[n][48]=df[n][49]=''
        df[n][50] = df[n][50] +' '+df[n][51] 
        df[n][51]=''
        df[n][52] = df[n][52] +' '+df[n][53] +' '+df[n][54] +' '+df[n][55] 
        df[n][53] = df[n][54] =df[n][55] =''
        df[n][56] = df[n][56] +' '+df[n][57] +' '+df[n][58] +' '+df[n][59] +' '+df[n][60] +' '+df[n][61] 
        df[n][57] = df[n][58] =df[n][59] =df[n][60] =df[n][61]=''
    df[1][15]=df[1][17]
    df[1][16]=df[1][18]
    for n in range(len(df)):
        if df[0][n] == '':
            df=df.drop(n)
    df = df.reset_index(drop = True)
    df[3][0] = df[3][0]+ ' '+df[3][1]+ ' '+df[3][2]+ ' '+df[3][3] + ' ' + df[3][4] + ' '+df[3][5]
    df[3][1]=df[3][2]=df[3][3]=df[3][4]=df[3][5]=df[3][6]=df[3][0]
    df[4][0] = df[4][0]+ ' '+df[4][1]+ ' '+df[4][2]+ ' '+df[4][3] + ' ' + df[4][4] 
    df[4][1]=df[4][2]=df[4][3]=df[4][4]=df[4][5]=df[4][6]=df[4][0]
    df[0][10] = df[0][10] +df[0][11]
    df[0][11] =''
    df[2][1]=df[2][2]=df[2][3]=df[2][4]=df[2][5]=df[2][0]
    df=df.drop(11)
    df = df.reset_index(drop = True)
    colname2id['risk'].remove(3)
    colname2id['impact']=[2,3]
    id2colname[3] =['impact']
    
    return df,colname2id,id2colname

In [21]:
#if country == 'algeria' and year==2014:
def algeria_2014(df,colname2id,id2colname):
    df = df.drop(0)
    df = df.drop(7)
    df = df.drop(10)
    df = df.drop(14)
    df = df.reset_index(drop = True)
    df[1][0]=df[0][0] +'&&'+df[1][0]
    df[1][1]=df[0][0] +'&&'+df[1][1]
    df[1][2]=df[0][2] +'&&'+df[1][2]
    df[1][3]=df[0][3] +'&&'+df[1][3]
    df[1][4]=df[0][3] +'&&'+df[1][4]
    del df[0]
    df[3][4]=df[3][3]
    df[3][5]=df[3][3]
    id2colname[1] = ['risk']
    id2colname[2] = ['likelihood']
    id2colname[3] = ['impact']
    colname2id['risk'] = [1]
    colname2id['likelihood'] = [2]
    colname2id['impact'] = [3]
    return df,colname2id,id2colname

In [22]:
#if country == 'hungary' and year == 2015:
def hungary_2015(df,colname2id,id2colname):
    policy_content = df[0][7]
    policy_special = []
    df = df.drop(6)
    df = df.drop(7)
    for n in range(len(df)):
        policy_special.append(policy_content)
    df[4] = policy_special
    del df[0]
    del id2colname[0]
    id2colname[1] = ['risk']
    id2colname[2] = ['likelihood']
    id2colname[3] = ['impact']
    id2colname[4] = ['policy']
    colname2id['risk'] = [1]
    colname2id['likelihood'] = [2]
    colname2id['impact'] = [3]
    colname2id['policy'] = [4]
    return df,colname2id,id2colname

In [23]:
#if country == 'hungary' and year == 2014
def hungary_2014(df,colname2id,id2colname):
    lis=[4,5]
    for n in lis:
        df[0][n] = df[1][n]
        df[1][n] = df[2][n]
        df[2][n] = df[3][n]
    del df[3]
    policy_content = df[0][7]
    policy_special = []
    df = df.drop(6)
    df = df.drop(7)
    for n in range(len(df)):
        policy_special.append(policy_content)
    df[col_num] = policy_special
    id2colname[0] = ['risk']
    id2colname[1] = ['likelihood']
    id2colname[2] = ['impact']
    id2colname[4] = ['policy']
    colname2id['risk'] = [0]
    colname2id['likelihood'] = [1]
    colname2id['impact'] = [2]
    colname2id['policy'] = [4]
    return df,colname2id,id2colname

In [24]:
#if country == 'colombia' and year == 2014:
def colombia_2014(df,colname2id,id2colname):
    df[0][3]=df[0][2]+'&&'+df[1][3]
    df[0][4]=df[0][2]+'&&'+df[1][4]
    df[0][8] = df[0][7]+'&&'+df[1][8]
    df[0][9] = df[0][7]+'&&'+df[1][9]
    df=df.drop(2)
    df=df.drop(7)
    df = df.reset_index(drop = True)
    list = [2,3,6,7]
    for n in list:
        df[1][n]=df[2][n]
        df[2][n]=df[3][n]
        df[3][n]=df[4][n]
    del df[4]
    colname2id['risk'] = [0]
    colname2id['likelihood'] = [1]
    colname2id['impact'] = [2]
    colname2id['policy'] = [3]
    id2colname[0]=['risk']
    id2colname[1]=['likelihood']
    id2colname[2]=['impact']
    id2colname[3]=['policy']
    return df,colname2id,id2colname

In [25]:
#if country == 'algeria' and year == 2017:
def algeria_2017(df,colname2id,id2colname):
    df[0][1]=df[0][0]+'&&'+df[1][1]
    df[0][3]=df[0][2]+'&&'+df[1][3]
    df[0][4]=df[0][2]+'&&'+df[1][4]
    df[0][6] = df[0][5]+'&&'+df[1][6]
    df[0][8] = df[0][7]+'&&'+df[1][8]
    df= df.drop(0)
    df= df.drop(2)
    df= df.drop(5)
    df= df.drop(7)
    del df[1]
    df = df.reset_index(drop = True)
    df[5][1] = df[5][0]
    df[5][2] = df[5][0]
    df[5][3] = df[5][0]
    colname2id['risk'] = [0]
    colname2id['likelihood'] = [2]
    colname2id['impact'] = [3,4]
    colname2id['policy'] = [5]
    id2colname[0]=['risk']
    id2colname[2]=['likelihood']
    id2colname[3]=['impact']
    id2colname[4]=['impact']
    id2colname[5]=['policy']
    return df,colname2id,id2colname

In [26]:
#if country == 'algeria' and year == 2016:
def algeria_2016(df,colname2id,id2colname):
    df[0][2]=df[0][1]+'&&'+df[1][2]
    df[0][3]=df[0][1]+'&&'+df[1][3]
    df[0][5] = df[0][4]+'&&'+df[1][5]
    df[0][7] = df[0][6]+'&&'+df[1][7]
    df=df.drop(1)
    df=df.drop(4)
    df=df.drop(6)
    df = df.reset_index(drop = True)
    for n in range(1,len(df)):
        df[1][n]=df[2][n]
        df[2][n]=df[3][n]
        df[3][n]=df[4][n]
        df[4][n]=df[5][n]
    del df[5]
    df[4][2] = df[4][1]
    colname2id['risk'] = [0]
    colname2id['likelihood'] = [1]
    colname2id['impact'] = [2,3]
    colname2id['policy'] = [4]
    id2colname[0]=['risk']
    id2colname[1]=['likelihood']
    id2colname[2]=['impact']
    id2colname[3]=['impact']
    id2colname[4]=['policy']
    return df,colname2id,id2colname

In [27]:
def albania_2014(df):
    df[0][2] = df[0][3]=df[0][1]
    df[1][1] = df[1][0] +'&&' +df[1][1] +' ' + df[1][2] +df[1][3]
    df[1][2] = df[1][3]=df[1][1]
    df[2][1] = df[2][0] + '&&' + df[2][1]
    df[2][2] = df[2][3] =df[2][1]
    df[1][5] = df[1][4] +'&&' +df[1][5] +' ' + df[1][6] +df[1][7]
    df[1][6] = df[1][7]=df[1][5]
    df[2][5] = df[2][4] + '&&' + df[2][5]
    df[2][6] = df[2][7] =df[2][5]
    df[0][11]= df[0][9]
    df[1][9] = df[1][8] +'&&' +df[1][9] +' ' + df[1][11] +df[1][12]
    df[1][10] = df[1][11]=df[1][9]
    df[2][9] = df[2][8] +'&&' +df[2][9] +' ' + df[2][10] + ' ' +df[2][13] +df[2][14]
    df[2][10] = df[2][11]=df[2][9]
    df[1][16] = df[1][15] +'&&' +df[1][16] 
    df[2][16] = df[2][15] +'&&' +df[2][16] 
    df[1][18] = df[1][17] +'&&' +df[1][18] 
    df[2][18] = df[2][17] +'&&' +df[2][18] 
    df=df.drop(0)
    df=df.drop(4)
    df=df.drop(8)
    df=df.drop(12)
    df=df.drop(13)
    df=df.drop(14)
    df=df.drop(15)
    df=df.drop(17)
    df = df.reset_index(drop = True)
    return df

In [55]:
def malta_2017(df,colname2id,id2colname):
    if len(policy_n) != 0:
        repeat = []
        repeat.append(policy_n[0])
        for n in range(len(policy_n)):
            if n != len(policy_n) -1 :
                repeat.append(policy_n[n+1] - policy_n[n])
        repeat[0] = repeat[0] + 1
        new_list = []
        for n in range(len(repeat)):
            for i in range(repeat[n]):
                new_list.append(df[0][policy_n[n]])
        df[col_num] = new_list
        df = df.drop(policy_n)
        index = []
        for n in range(len(df)):
            index.append(n)
        df = df.reset_index(drop = True)
        id2colname[0].remove('policy')
        id2colname[col_num ] = ['policy']
        colname2id['policy'] = [col_num ]
    return df,colname2id,id2colname

In [120]:
def new_zealand_2016(df,colname2id,id2colname):
    df= Insert_row(6,df,[df[0][4],'','','',''])
    for i in range(col_num):
        for n in range(len(df)):
            if df[i][n] == 'None' or type(df[i][n]) != str:
                df[i][n] = ''

    for n in range(len(df)):
        if n%2 ==0:
            df[0][n+1] = df[0][n] +'&&' +df[0][n+1]
            df=df.drop(n)
    colname2id['risk'] = [0]
    colname2id['likelihood'] = [1]
    colname2id['horizon'] = [2]
    colname2id['impact'] = [3]
    colname2id['policy'] = [4]
    id2colname[0] = ['risk']
    id2colname[1] = ['likelihood']
    id2colname[2] = ['horizon']
    id2colname[3] = ['impact']
    id2colname[4] = ['policy']
    return df,colname2id,id2colname

In [129]:
def antigua_and_barbuda(df,colname2id,id2colname):
    df=df.drop(5)
    df = df.reset_index(drop = True)
    df[1][1]=df[1][2] = df[1][0] +'&&'+ df[1][1]
    df[1][4]=df[1][5]=df[1][6] = df[1][3] +'&&'+ df[1][4]
    df[1][8] = df[1][7] +'&&'+ df[1][8]
    df[1][10] = df[1][9] + '&&'+df[1][10]
    df[1][12] = df[1][11] + '&&'+df[1][12]
    df=df.drop(0)
    df = df.reset_index(drop = True)
    return df,colname2id,id2colname

In [225]:
def fiji_2018(df,colname2id,id2colname):
    impact = []
    for n in range(len(df)):
        if n%2==0:
            impact.append(df[1][n])
            df=df.drop(n)
    df = df.reset_index(drop = True)
    df[4]=impact
    colname2id['risk'] = [1]
    colname2id['likelihood'] = [0]
    colname2id['policy'] = [3]
    colname2id['impact'] = [4]
    id2colname[1] = ['risk']
    id2colname[0] = ['likelihood']
    id2colname[3] = ['policy']
    id2colname[4] = ['impact']
    return df,colname2id,id2colname

In [276]:
def sri_lanka_2014(df,colname2id,id2colname):
    df[2][0] = df[2][0] +' '+df[2][1]+' '+df[2][2] +df[2][3]
    df[2][1]=df[2][2]=df[2][3]=''
    df[3][0] = df[3][0] +' '+df[3][1]+' '+df[3][2] +df[3][3]+df[3][4]+df[3][5]
    df[3][1]=df[3][2]=df[3][3]=df[3][4]=df[3][5]=''
    df[0][6]=df[0][6]+' '+df[0][7]
    df[0][7]=''
    df[2][6] = df[2][6] +' '+df[2][7]+' '+df[2][8] +df[2][9]
    df[2][7]=df[2][8]=df[2][9]=''
    df[3][6] = df[3][6] +' '+df[3][7]+' '+df[3][8] +df[3][9]
    df[3][7]=df[3][8]=df[3][9]=''
    df[0][10]=df[0][10]+' '+df[0][11]
    df[0][11]=''
    df[2][10] = df[2][10] +' '+df[2][11]+' '+df[2][12] +df[2][13]
    df[2][11]=df[2][12]=df[2][13]=''
    df[3][10] = df[3][10] +' '+df[3][11]+' '+df[3][12] +df[3][13]
    df[3][11]=df[3][12]=df[3][13]=''
    df[0][14]=df[0][14]+' '+df[0][15]+' '+df[0][16]+' '+df[0][17]+' '+df[0][18]
    df[0][15]=df[0][16]=df[0][17]=df[0][18]=''
    df[2][14] = df[2][14] +' '+df[2][15]+' '+df[2][16] 
    df[2][15]=df[2][16]=''
    df[3][14] = df[3][14] +' '+df[3][15]+' '+df[3][16] +df[3][17]
    df[3][15]=df[3][16]=df[3][17]=''
    df[0][19] = df[0][19] +' '+df[0][20]+' '+df[0][21] 
    df[0][20]=df[0][21]=''
    df[2][19]=df[2][19]+' '+df[2][20]+' '+df[2][21]+' '+df[2][22]+' '+df[2][23]
    df[2][20]=df[2][21]=df[2][22]=df[2][23] = ''
    df[3][19] = df[3][19] +' '+df[3][20]+' '+df[3][21] 
    df[3][20]=df[3][21]=''
    df[0][24]=df[0][24]+' '+df[0][25]+' '+df[0][26]+' '+df[0][27]
    df[0][25]=df[0][26]=df[0][27]=''
    df[2][24]=df[2][24]+' '+df[2][25]+' '+df[2][26]+' '+df[2][27]
    df[2][25]=df[2][26]=df[2][27]=''
    df[3][24]=df[3][24]+' '+df[3][25]+' '+df[3][26]+' '+df[3][27]
    df[3][25]=df[3][26]=df[3][27]=''
    for n in range(4):
        df[n][1]=df[n][6]
        df[n][2]=df[n][10]
        df[n][3]=df[n][14]
        df[n][4]=df[n][19]
        df[n][5]=df[n][24]
    for n in range(6,28):
        df=df.drop(n)
    df = df.reset_index(drop = True)
    return df,colname2id,id2colname

In [315]:
def iran_2018(df,colname2id,id2colname):
    name=df[0][0]
    df=df.drop(0)
    df = df.reset_index(drop = True)
    del colname2id['impact']
    for n in range(10):
        if n%2==1:
            df[0][n]=name+'&&'+df[0][n]
    colname2id['impact']=[3]
    #id2colname[2].remove('impact')
    id2colname[3]=['impact']
    return df,colname2id,id2colname

In [298]:
def south_korea_2014(df,colname2id,id2colname):
    df=df.drop(0)
    df = df.reset_index(drop = True)
    df=Insert_row(2,df,[df[0][1],df[1][0],df[2][0]])
    df[1][3]=df[1][1]
    policy = []
    for n in range(16):
        policy.append(df[0][16])
    df=df.drop(16)
    df = df.reset_index(drop = True)
    df[3]=policy
    colname2id['policy']=[3]
    id2colname[3]=['policy']
    return df,colname2id,id2colname

In [324]:
def sweden_2016(df,colname2id,id2colname):
    policy=[]
    for n in range(len(df)):
        if n%2==1:
            policy.append(df[1][n])
            df=df.drop(n)
    df[3]=policy
    df = df.reset_index(drop = True)
    colname2id['policy']=[3]
    id2colname[3]=['policy']
    return df,colname2id,id2colname

In [313]:
def solomon_islands_2018(df,colname2id,id2colname):
    impact=[]
    for n in range(len(df)):
        if n%2==0:
            impact.append(df[1][n])
            df[1][n]=''
    impact1=[]
    for n in impact:
        impact1.append(n)
        impact1.append(n)
    df[4]=impact1
    colname2id['likelihood']=[0]
    colname2id['risk']=[1]
    colname2id['impact']=[4]
    id2colname[4]=['impact']
    id2colname[0]=['likelihood']
    id2colname[1]=['risk']
    return df,colname2id,id2colname